In [1]:
import os
import seaborn as sns
path = '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Analysis/Figures'
import sys
sys.path.insert(1, '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Data Collection')
from configs import *
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.axisartist.axislines import SubplotZero
from pylab import text
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://'+user+':'+passwd+'@'+ip+':3306/'+schema1)

In [2]:
df = pd.read_csv('/Users/connormcdonald/Desktop/Masters/MIT807/Data/twitterMAU.csv')
df['period']  = pd.to_datetime(df['period'])

In [3]:
df1 = df.iloc[:38,:]

In [4]:
import matplotlib.dates as mdates


degree = 3
y = np.array(df1['mau'].to_list())
X = np.array(mdates.date2num(df1['period']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()

# print(results.summary().as_latex())

In [5]:
X = mdates.date2num(df['period'])

In [6]:
x_fit = mdates.date2num(df['period'])
y_fit = [model(_x) for _x in x_fit]

In [7]:
def dformat(d):
    return d.strftime('%Y-%m')

def yformat(d):
    return d.strftime('%Y')

In [8]:
df['y_fit'] = y_fit
df['combined_mau'] = np.where(df["mau"].isnull(), df["y_fit"], df["mau"] )*1000000
df['period_formatted'] = df['period'].apply(dformat)

In [9]:
stmt = '''SELECT DATE_FORMAT(date, \'%Y-%m\') as date, 
COUNT(1) AS tweets,
SUM(like_count) as likes,
SUM(quote_count) as quotes,
SUM(reply_count) as comments,
SUM(retweet_count) as retweets
FROM social.blockchain 
WHERE date < \'2018-01-01\'
group by DATE_FORMAT(date, \'%Y-%m\') 
ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC'''
df2018 = pd.read_sql(stmt, con=engine)



In [10]:
stmt = '''SELECT DATE_FORMAT(date, \'%Y-%m\') as date, 
COUNT(1) AS tweets,
SUM(like_count) as likes,
SUM(quote_count) as quotes,
SUM(reply_count) as comments,
SUM(retweet_count) as retweets
FROM social.blockchain 
WHERE date >= \'2018-01-01\' AND date < \'2019-01-01\'
group by DATE_FORMAT(date, \'%Y-%m\') 
ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC'''
df2019 = pd.read_sql(stmt, con=engine)

In [11]:
stmt = '''SELECT DATE_FORMAT(date, \'%Y-%m\') as date, 
COUNT(1) AS tweets,
SUM(like_count) as likes,
SUM(quote_count) as quotes,
SUM(reply_count) as comments,
SUM(retweet_count) as retweets
FROM social.blockchain 
WHERE date >= \'2019-01-01\' AND date < \'2020-06-01\'
group by DATE_FORMAT(date, \'%Y-%m\') 
ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC'''
df2020 = pd.read_sql(stmt, con=engine)

In [12]:
stmt = '''SELECT DATE_FORMAT(date, \'%Y-%m\') as date, 
COUNT(1) AS tweets,
SUM(like_count) as likes,
SUM(quote_count) as quotes,
SUM(reply_count) as comments,
SUM(retweet_count) as retweets
FROM social.blockchain 
WHERE date >= \'2020-06-01\'
group by DATE_FORMAT(date, \'%Y-%m\') 
ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC'''
df2021 = pd.read_sql(stmt, con=engine)

In [13]:
df2 = df2018.append(df2019, ignore_index=True)
df2 = df2.append(df2020, ignore_index=True)
df2 = df2.append(df2021, ignore_index=True)
df2

/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_37993/2731392987.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2018.append(df2019, ignore_index=True)
/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_37993/2731392987.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df2020, ignore_index=True)
/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_37993/2731392987.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df2021, ignore_index=True)


,date,tweets,likes,quotes,comments,retweets
0,2011-03,1,0.0,0.0,0.0,0.0
1,2011-04,2,0.0,0.0,0.0,1.0
2,2011-05,11,1.0,0.0,0.0,1.0
3,2011-06,11,3.0,0.0,1.0,1.0
4,2011-07,6,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
125,2021-08,459401,2905291.0,356300.0,543299.0,1427199.0
126,2021-09,438051,3454590.0,325614.0,595230.0,1611273.0
127,2021-10,570588,4378248.0,407846.0,821868.0,2032221.0
128,2021-11,609901,4863002.0,353633.0,901950.0,2198923.0


In [14]:
df3 = pd.merge(df, df2, left_on='period_formatted', right_on='date', how='inner')

df3['tweets_p_user'] = df3['tweets']/df3['combined_mau']

df3['likes_p_user'] = df3['likes']/df3['combined_mau']
df3['comments_p_user'] = df3['comments']/df3['combined_mau']
df3['quotes_p_user'] = df3['quotes']/df3['combined_mau']
df3['retweets_p_user'] = df3['retweets']/df3['combined_mau']

df3['total_engagement'] = df3['likes_p_user'] + df3['comments_p_user'] + df3['quotes_p_user'] + df3['retweets_p_user']
df3['total_engagement_og'] = df3['likes'] + df3['comments'] + df3['quotes'] + df3['retweets']

In [15]:
correlation_df = df3
correlation_df['date']  = pd.to_datetime(correlation_df['date'])

correlation_pre = correlation_df[correlation_df.period_formatted < '2020-12-01']
correlation_post = correlation_df[correlation_df.period_formatted >= '2020-12-01']


column_1 = correlation_df["tweets"]
column_2 = correlation_df["total_engagement_og"]
correlation = column_1.corr(column_2)
print(f"total: {correlation}")


column_1 = correlation_pre["tweets"]
column_2 = correlation_pre["total_engagement_og"]
correlation = column_1.corr(column_2)
print(f"pre: {correlation}")


column_1 = correlation_post["tweets"]
column_2 = correlation_post["total_engagement_og"]
correlation = column_1.corr(column_2)
print(f"post: {correlation}")

total: 0.8231709355998323
pre: 0.9839317472242228
post: 0.9834953136801926


In [16]:
from scipy.stats.stats import pearsonr
corr = []
c = []
p = []
for i in range(2, len(correlation_df.index)):
    temp_df = correlation_df.iloc[:i,:]
    tweets = temp_df.tweets
    engagement = temp_df.total_engagement_og
    correlation = tweets.corr(engagement)
    corr.append(correlation)
    x = pearsonr(engagement, tweets)
    c.append(x[0])
    p.append(x[1])


/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_37993/1982449608.py:1: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [43]:
print(corr)

[0.9999999999999999, 0.9762210399274297, 0.9907747849205183, 0.9931541988391678, 0.9157437185514745, 0.9730441806376738, 0.9298510831080337, 0.9191849874225453, 0.9189547095731139, 0.9109495430157009, 0.9861760199469932, 0.9951342930868744, 0.9961272421871855, 0.9957077164020064, 0.9750933272931203, 0.9468170242972216, 0.9631822769798077, 0.9466386748519838, 0.9787909575513023, 0.9349696533300998, 0.9488667990926788, 0.9670626783107076, 0.9765355940523974, 0.9833925042909903, 0.949935832007426, 0.95839511176884, 0.9752872136993905, 0.9821567576987631, 0.9876322006135541, 0.9890081672466347, 0.9883024532469883, 0.9881098331837077, 0.9880060751521628, 0.9867973477021278, 0.9852987856971012, 0.9839317472242228, 0.9403841127992422, 0.9161689268271157, 0.8636090444753136]


In [17]:
#corr plot
line = pd.to_datetime('2014-09-01')
line2 = pd.to_datetime('2018-03-01')
line3 = pd.to_datetime('2020-12-01')
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Correlation')
ax.set_ylim([0, 1])
plt.plot(correlation_df.loc[2:].date, corr, c = 'black', linewidth = 1)
# plt.plot(correlation_df.loc[2:].date, p, c = 'green', linewidth = 1)
plt.axvline(x=line, c = '#E31B23', linewidth = 1, linestyle ="--", label= 'POI-A')
plt.axvline(x=line3, c = '#FF9A00', linewidth = 1, linestyle ="--", label= 'POI-B')
plt.axvline(x=line2, c = '#999999', linewidth = 1, linestyle ="--", label= 'POI-C')
plt.legend(loc='lower left')
plt.savefig(os.path.join(path, 'bc_correleation.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()


In [18]:
#corr plot
line = pd.to_datetime('2014-09-01')
line2 = pd.to_datetime('2018-03-01')
line3 = pd.to_datetime('2020-12-01')
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Correlation')
ax.set_ylim([-0.2, 0.8])
plt.plot(correlation_df.loc[2:].date, p, c = '#339898', linewidth = 1)
plt.axvline(x=line, c = '#E31B23', linewidth = 1, linestyle ="--", label= 'POI-A')
plt.axvline(x=line3, c = '#FF9A00', linewidth = 1, linestyle ="--", label= 'POI-B')
plt.axvline(x=line2, c = '#999999', linewidth = 1, linestyle ="--", label= 'POI-C')
plt.legend(loc='lower left')
plt.savefig(os.path.join(path, 'bc_pval.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [19]:
degree = 3
y = np.array(df3['tweets_p_user'].to_list())
X = np.array(mdates.date2num(df3['date']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()


X = mdates.date2num(df3['date'])

x_fit = mdates.date2num(df3['date'])
y_fit = [model(_x) for _x in x_fit]

df3['y_fit_2'] = y_fit

In [15]:
print(results.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &        y         & \textbf{  R-squared:         } &     0.550   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.538   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     46.38   \\
\textbf{Date:}             & Mon, 19 Sep 2022 & \textbf{  Prob (F-statistic):} &  4.45e-08   \\
\textbf{Time:}             &     17:29:09     & \textbf{  Log-Likelihood:    } &    252.34   \\
\textbf{No. Observations:} &          40      & \textbf{  AIC:               } &    -500.7   \\
\textbf{Df Residuals:}     &          38      & \textbf{  BIC:               } &    -497.3   \\
\textbf{Df Model:}         &           1      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                   & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$> |$t$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\midru

In [16]:
df3.head()

,period,mau,y_fit,combined_mau,period_formatted,date,tweets,likes,quotes,comments,retweets,tweets_p_user,likes_p_user,comments_p_user,quotes_p_user,retweets_p_user,total_engagement,y_fit_2
0,2011-03-31,68.0,80.980447,68000000.0,2011-03,2011-03,1,0.0,0.0,0.0,0.0,1.470588e-08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000210
1,2011-06-30,85.0,98.278006,85000000.0,2011-06,2011-06,11,3.0,0.0,1.0,1.0,1.294118e-07,3.529412e-08,1.176471e-08,0.0,1.176471e-08,5.882353e-08,0.000109
2,2011-09-30,101.0,115.131254,101000000.0,2011-09,2011-09,24,1.0,0.0,1.0,6.0,2.376238e-07,9.900990e-09,9.900990e-09,0.0,5.940594e-08,7.920792e-08,0.000027
3,2011-12-31,117.0,131.348972,117000000.0,2011-12,2011-12,65,8.0,0.0,4.0,22.0,5.555556e-07,6.837607e-08,3.418803e-08,0.0,1.880342e-07,2.905983e-07,-0.000038
4,2012-03-31,138.0,146.767500,138000000.0,2012-03,2012-03,78,9.0,0.0,3.0,25.0,5.652174e-07,6.521739e-08,2.173913e-08,0.0,1.811594e-07,2.681159e-07,-0.000087


In [20]:
curr_max_freq = 0
curr_max_eng = 0
for i in range(len(df3.index)-1):
    a = df3.tweets_p_user[i]
    b = df3.tweets_p_user[i+1]

    c = df3.total_engagement[i]
    d = df3.total_engagement[i+1]


    if a > b and i !=0 and c > d and a > curr_max_freq and c > curr_max_eng and c > a:
        print(df3.period_formatted[i])
        curr_max_freq = df3.tweets_p_user[i]
        curr_max_eng = df3.total_engagement[i]

2018-03


In [21]:
#when engagement overtook tweets

for i in range(len(df3.index)):
    a = df3.tweets_p_user[i]
    b = df3.total_engagement[i]

    if a < b: 
        print(df3.period_formatted[i])

2014-09
2014-12
2015-03
2015-06
2015-09
2015-12
2016-03
2016-06
2016-09
2016-12
2017-03
2017-06
2017-09
2017-12
2018-03
2018-06
2018-09
2018-12
2019-03
2020-03
2020-06
2020-09
2020-12
2021-03
2021-06
2021-09
2021-12


In [25]:
#ML plot
df3['date']  = pd.to_datetime(df3['date'])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 3]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Monthly Tweets Per Active User')
plt.plot(df3['date'], df3['tweets_p_user'], c = '#339898', linewidth = 1, label = 'Tweets')
plt.plot(df3['date'], df3['y_fit_2'], c = '#E31B23', linewidth = 1, linestyle ="--", label= 'Polynomial Trend Line\n ($R^2$ = 0.525)')
plt.legend(loc='upper left')
plt.savefig(os.path.join(path, 'BC_normalised_tweets.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()



In [22]:
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300


#define colors to use in chart
color_map = ['#339898', '#E31B23', '#999999', '#FF9A00']

#create area chart
plt.stackplot(df3['date'], df3['likes_p_user'], df3['comments_p_user'], df3['quotes_p_user'],df3['retweets_p_user'],
              labels=['Likes', 'Comments', 'Quotes', 'Retweets'],
              colors=color_map,
              alpha=0.6)
plt.plot(df3['date'], df3['tweets_p_user'], c = 'black', linewidth = 1,label = 'Tweets')

#add legend
plt.legend(loc='upper left')

#add axis labels
plt.xlabel('Time')
plt.ylabel('Monthly Metrics Per Active User')
plt.show()
plt.savefig(os.path.join(path, 'bc_normalised_area.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()



<ipython-input-22-fc1f9c3940bd>:25: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [19]:
#ML plot
line = pd.to_datetime('2014-09-01')
line2 = pd.to_datetime('2018-03-01')
line3 = pd.to_datetime('2020-12-01')
# line4 = pd.to_datetime('2014-06-01')
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Monthly Tweets Per Active User')
plt.plot(df3['date'], df3['tweets_p_user'], c = 'black', linewidth = 1, label = 'Tweets')
plt.axvline(x=line, c = '#E31B23', linewidth = 1, linestyle ="--", label= 'PoI-A')
plt.axvline(x=line2, c = '#999999', linewidth = 1, linestyle ="--", label= 'PoI-B')
plt.axvline(x=line3, c = '#FF9A00', linewidth = 1, linestyle ="--", label= 'PoI-C')
# plt.axvline(x=line4, c = '#999999', linewidth = 1, linestyle ="--")
# plt.plot(df3['date'], df3['y_fit_2'], c = '#E31B23', linewidth = 1, linestyle ="--", label= 'Polynomial Trend Line (R^2 = 0.686)')
plt.legend(loc='upper left')
plt.savefig(os.path.join(path, 'bc_shift.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()



In [2]:
df = pd.read_csv('/Users/connormcdonald/Desktop/Masters/MIT807/Data/academic_results.csv')

In [3]:
year = ['2010-12-31', '2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']
papers = [2, 0, 1, 2, 12, 41, 183, 824, 3028, 6070, 8044, 9939]
df['papers'] = papers

In [4]:
df['scopus_norm'] = df['papers']/df['scopus_results']

In [5]:
df

,period,scholar_results,scopus_results,papers,scopus_norm
0,2010,4510000,2478126,2,8.070615e-07
1,2011,4590000,2638921,0,0.000000e+00
2,2012,4670000,2775816,1,3.602544e-07
3,2013,4360000,2901649,2,6.892632e-07
4,2014,4640000,2942515,12,4.078144e-06
5,2015,4710000,2953613,41,1.388130e-05
6,2016,3040000,3055033,183,5.990115e-05
7,2017,3450000,3151892,824,2.614303e-04
8,2018,3960000,3283566,3028,9.221682e-04
9,2019,3500000,3437602,6070,1.765766e-03


In [6]:
import matplotlib.dates as mdates
degree = 3
y = np.array(df['scopus_norm'].to_list())
X = np.array(mdates.date2num(df['period']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()


X = mdates.date2num(df['period'])

x_fit = mdates.date2num(df['period'])
y_fit = [model(_x) for _x in x_fit]

df['y_fit_2'] = y_fit

In [45]:
print(results.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &        y         & \textbf{  R-squared:         } &     0.946   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.946   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &       nan   \\
\textbf{Date:}             & Sun, 18 Sep 2022 & \textbf{  Prob (F-statistic):} &      nan    \\
\textbf{Time:}             &     08:18:18     & \textbf{  Log-Likelihood:    } &    84.163   \\
\textbf{No. Observations:} &          12      & \textbf{  AIC:               } &    -166.3   \\
\textbf{Df Residuals:}     &          11      & \textbf{  BIC:               } &    -165.8   \\
\textbf{Df Model:}         &           0      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                   & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$> |$t$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\midru

/Users/connormcdonald/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [8]:
#ML plot
# df['period']  = pd.to_datetime(df['period'])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 3]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Proportion of Blockchain Publications')
plt.plot(df['period'], df['scopus_norm'], c = '#339898', linewidth = 1, label ='Publications')
plt.plot(df['period'], df['y_fit_2'], c = '#E31B23', linewidth = 1, linestyle ="--", label= 'Polynomial Trend Line ($R^2$ = 0.946)')
plt.legend(loc='upper left')
plt.savefig(os.path.join(path, 'BC_normalised_pubs.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [28]:
stmt = 'SELECT text, YEAR(date) as year FROM social.machine_learning_only'

df = pd.read_sql(stmt, con=engine)

KeyboardInterrupt: 

In [ ]:
df.head()

In [ ]:
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax = sns.violinplot(x="year", y="char_count", data=df, linewidth=1)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Year')
plt.ylabel('Character Count Per Tweet')
plt.savefig(os.path.join(path, 'char_count.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [13]:
#Innovation Trigger

stmt = '''
SELECT A.text
FROM social.blockchain A
WHERE A.date >= \'2014-08-01\' and A.date <= \'2014-09-30\'
'''

df = pd.read_sql(stmt, con=engine)

In [25]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}
          
## Defining set containing all stopwords in english.
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

# nltk
from nltk.stem import WordNetLemmatizer
import re

def preprocess(textdata):
    processedText = []
    
    # Create Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for tweet in textdata:
        tweet = tweet.lower()
        
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,' URL',tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' USER', tweet)        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # Checking if the word is a stopword.
            #if word not in stopwordlist:
            if len(word)>1:
                # Lemmatizing the word.
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [15]:
import time
text = df['text']
t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 1 seconds


In [30]:
from wordcloud import WordCloud
from nltk.corpus import stopwords

#Total Wordcloud
allwords = ' '.join([tweets for tweets in processedtext]) 
stop_words = stopwords.words('english')
stop_words.append('blockchain')
# stop_words.append('ml')
# stop_words.append('machinelearning')
# stop_words.append('learning')
# stop_words.append('ai')
# stop_words.append('artificial')
# stop_words.append('intelligence')
# stop_words.append('artificialintelligence')

In [17]:
plt.rcParams['figure.dpi'] = 300
# plt.rcParams["figure.figsize"] = [8, 4]
wordCloud = WordCloud(
    font_path='/Library/Fonts/cmunrm.ttf',
    width=800, 
    height=800,
    background_color="white",
    max_font_size = 200,
    collocation_threshold = 50,
    stopwords=stop_words,
    random_state=1,
    colormap="gist_heat").generate(allwords)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.savefig(os.path.join(path, 'bc_innovation_trigger.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [18]:
#peak

stmt = '''
SELECT A.text
FROM social.blockchain A
WHERE A.date >= \'2018-02-01\' and A.date <= \'2018-03-31\'
'''

df = pd.read_sql(stmt, con=engine)

In [19]:
import time
text = df['text']
t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 128 seconds


In [20]:
allwords = ' '.join([tweets for tweets in processedtext])
plt.rcParams['figure.dpi'] = 300
# plt.rcParams["figure.figsize"] = [8, 4]
wordCloud = WordCloud(
    font_path='/Library/Fonts/cmunrm.ttf',
    width=800, 
    height=800,
    background_color="white",
    max_font_size = 200,
    collocation_threshold = 50,
    stopwords=stop_words,
    random_state=1,
    colormap="gist_heat").generate(allwords)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.savefig(os.path.join(path, 'bc_peak.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [26]:
#trough

stmt = '''
SELECT A.text
FROM social.blockchain A
WHERE A.date >= \'2019-11-01\' and A.date <= \'2019-12-31\'
'''

df = pd.read_sql(stmt, con=engine)

In [28]:
text = df['text']
# t = time.time()
processedtext = preprocess(text)
# print(f'Text Preprocessing complete.')
# print(f'Time Taken: {round(time.time()-t)} seconds')

In [31]:
allwords = ' '.join([tweets for tweets in processedtext])
plt.rcParams['figure.dpi'] = 300
# plt.rcParams["figure.figsize"] = [8, 4]
wordCloud = WordCloud(
    font_path='/Library/Fonts/cmunrm.ttf',
    width=800, 
    height=800,
    background_color="white",
    max_font_size = 200,
    collocation_threshold = 50,
    stopwords=stop_words,
    random_state=1,
    colormap="gist_heat").generate(allwords)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.savefig(os.path.join(path, 'bc_trough.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()